# Equity Risk Premium

The Fama-French 5-factor [data-set](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html) has the Rm-Rf field denoting the market's excess return over the risk-free rate.

Cumulative excess returns over different rolling windows (5, 10, 20 yrs) should give us an idea of how equity investors have been compensated for the additional risk that they took.

Documentation for the Fama-French data-set on pluto: [python](https://plutopy.readthedocs.io/en/latest/FamaFrench.html), [R](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

In [ ]:
dailyExs <- famaFrench$FiveFactor3x2Daily() %>%
    filter(KEY_ID == 'MKT-RF') %>%
    select(TIME_STAMP, RET) %>%
    collect()
    
dailyExsXts <- xts(dailyExs$RET/100, dailyExs$TIME_STAMP)

print("Excess Returns: ")
print(head(dailyExsXts))
print(tail(dailyExsXts))

In [ ]:
plotRollingReturns <- function(numYears, retXts){
    numPer <- 220 * numYears
    retUnits <- "days"
    
    rollRet <- rollapply(retXts, numPer, Return.cumulative)
    rollRet <- 100*na.omit(rollRet)
    names(rollRet) <- c('RET')
    
    rollRetDf <- data.frame(rollRet)
    rollRetDf$T <- index(rollRet)

    firstDate <- first(index(rollRet))
    lastDate <- last(index(rollRet))
    xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)

    ggplot(rollRetDf, aes(x=T, y=RET)) +
        theme_economist() +
        geom_line() +
        scale_x_date(breaks = xAxisTicks) +
        labs(x='', y='cumulative return (%)', color='', 
             title=sprintf("Excess %d-year Rolling Returns (%s)", numYears, retUnits), 
             subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
        annotate("text", x=lastDate, y=min(rollRetDf$RET, na.rm=T), label = "@StockViz", 
                 hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)
}

In [ ]:
#5-year rolling returns
plotRollingReturns(5, dailyExsXts)

In [ ]:
#10-year rolling returns
plotRollingReturns(10, dailyExsXts)

In [ ]:
#20-year rolling returns
plotRollingReturns(20, dailyExsXts)